# Using the Python `chi` library

## Getting started

Before you do anything, you should first set up a few project variables. This is so your script knows which project to operate on. If you are only a member of one project, you can skip this step, as the project will be selected for you by default. You can also set the site you want to use via the `region_name` setting - this defaults to `CHI@UC`.

In [ ]:
import chi

chi.set('project_name', 'chameleon')    # Replace with your project name
chi.set('region_name', 'CHI@UC')        # Optional, defaults to 'CHI@UC'
chi.use_site('CHI@UC')                  # Authenticate to the CHI@UC site

## Launching a baremetal server

Starting a new baremetal server is easy to do.

In [ ]:
import chi.lease
import chi.server

# Create a reservation for 1 node
reservations = []
chi.lease.add_node_reservation(reservations, node_type='compute_skylake')

# Create a lease for 1 node
lease = chi.lease.create_lease("getting-started-lease", reservations=reservations)
lease
chi.lease.wait_for_active(lease['id'])  # Ensure lease has started

# Launch 1 server
reservation_id = chi.lease.get_node_reservation(lease['id'])
server = chi.server.create_server("getting-started-instance", reservation_id, image_name='CC-Ubuntu20.04')
server = chi.server.wait_for_active(server.id)  # Ensure server has started
server

## Connecting to your server

By default, the server will only have a private IP assigned. In order to SSH to the server, you should assign a public floating IP. There are are limited amount of public IPs available across the entire Chameleon testbed, so try to keep the amount of nodes with a public IP to a minimum! A common practice is to set up one node as a "login node" with a public IP, and logging in to that node to manage all of your project's nodes.

Once a public IP is assigned, you can set up a remote connection to the node.

In [ ]:
from chi.ssh import Remote

# Assign a public floating IP
ip = chi.server.associate_floating_ip(server.id)

# Create a remote connection
remote = Remote(ip)

You can use `remote.run` to run commands on the server directly from your Notebook.
A similar command is `remote.sudo`, which runs a command with escalated privileges.

### Example command: get CPU usage

In [ ]:
# Run some commands on your remote server
cpu_usage = remote.run("grep 'cpu ' /proc/stat | awk '{usage=($2+$4)*100/($2+$4+$5)} END {print usage}'", hide=True)

print('Current CPU usage is {}%'.format(float(cpu_usage.stdout)))